# **1917. Leetcodify Friends Recommendations**

**Table: Listens**
``` console
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| user_id     | int     |
| song_id     | int     |
| day         | date    |
+-------------+---------+
```
This table may contain duplicates (In other words, there is no primary key for this table in SQL).
Each row of this table indicates that the user user_id listened to the song song_id on the day day.
 

**Table: Friendship**
``` console
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user1_id      | int     |
| user2_id      | int     |
+---------------+---------+
```
In SQL,(user1_id, user2_id) is the primary key for this table.
Each row of this table indicates that the users user1_id and user2_id are friends.
Note that user1_id < user2_id.
 

Recommend friends to Leetcodify users. We recommend user x to user y if:

Users x and y are not friends, and
Users x and y listened to the same three or more different songs on the same day.
Note that friend recommendations are unidirectional, meaning if user x and user y should be recommended to each other, the result table should have both user x recommended to user y and user y recommended to user x. Also, note that the result table should not contain duplicates (i.e., user y should not be recommended to user x multiple times.).

Return the result table in any order.

The result format is in the following example.

 

**Example 1:**

**Input:** 
**Listens table:**
``` console
+---------+---------+------------+
| user_id | song_id | day        |
+---------+---------+------------+
| 1       | 10      | 2021-03-15 |
| 1       | 11      | 2021-03-15 |
| 1       | 12      | 2021-03-15 |
| 2       | 10      | 2021-03-15 |
| 2       | 11      | 2021-03-15 |
| 2       | 12      | 2021-03-15 |
| 3       | 10      | 2021-03-15 |
| 3       | 11      | 2021-03-15 |
| 3       | 12      | 2021-03-15 |
| 4       | 10      | 2021-03-15 |
| 4       | 11      | 2021-03-15 |
| 4       | 13      | 2021-03-15 |
| 5       | 10      | 2021-03-16 |
| 5       | 11      | 2021-03-16 |
| 5       | 12      | 2021-03-16 |
+---------+---------+------------+
```
**Friendship table:**
``` console
+----------+----------+
| user1_id | user2_id |
+----------+----------+
| 1        | 2        |
+----------+----------+
```
**Output:** 
``` console
+---------+----------------+
| user_id | recommended_id |
+---------+----------------+
| 1       | 3              |
| 2       | 3              |
| 3       | 1              |
| 3       | 2              |
+---------+----------------+
```
**Explanation:** 

Users 1 and 2 listened to songs 10, 11, and 12 on the same day, but they are already friends.

Users 1 and 3 listened to songs 10, 11, and 12 on the same day. Since they are not friends, we recommend them to each other.

Users 1 and 4 did not listen to the same three songs.

Users 1 and 5 listened to songs 10, 11, and 12, but on different days.

Similarly, we can see that users 2 and 3 listened to songs 10, 11, and 12 on the same day and are not friends, so we recommend them to each other.

**Solution-1:**

``` sql
# Write your MySQL query statement below

SELECT a.user_id AS user_id , b.user_id AS recommended_id
FROM Listens a
JOIN Listens b
ON a.user_id <> b.user_id AND a.song_id = b.song_id and a.day = b.day
WHERE (a.user_id, b.user_id) NOT IN( 
SELECT user1_id , user2_id from friendship
UNION 
SELECT user2_id , user1_id from friendship)
GROUP BY a.user_id, b.user_id, a.day
HAVING COUNT(DISTINCT a.song_id)>=3
ORDER by 1,2

**solution-2:**

``` sql
# Write your MySQL query statement below
WITH cte AS(
SELECT a.user_id AS user_id , b.user_id AS recommended_id
FROM Listens a
JOIN Listens b
ON a.user_id < b.user_id AND a.song_id = b.song_id and a.day = b.day
WHERE NOT EXISTS (SELECT user1_id, user2_id from friendship WHERE a.user_id = user1_id AND b.user_id = user2_id)
GROUP BY a.user_id, b.user_id, a.day
HAVING COUNT(DISTINCT a.song_id)>=3)

SELECT user_id, recommended_id FROM cte
UNION
SELECT recommended_id, user_id FROM cte


**Solution-3:**

``` sql
-- Write your PostgreSQL query statement below

-- step 1: find users listened to the sam song three or more different song on the same day

WITH tbl_song_more_than_2 AS(
SELECT 
	L.user_id, L1.user_id user_id1
FROM
	Listens L
INNER JOIN
	Listens L1 ON L.day = L1.day AND L.user_id <> L1.user_id AND L.song_id = L1.song_id
GROUP BY 
	L.user_id, L1.user_id, L.day
HAVING COUNT(DISTINCT L.song_id) > 2
)

-- step 2: filter users not friend => not in table "friendShip"

SELECT DISTINCT
	user_id user_id,
	user_id1 recommended_id
FROM
	tbl_song_more_than_2 C WHERE NOT EXISTS (SELECT 1 FROM Friendship F WHERE  (C.user_id = F.user1_id AND C.user_id1 = F.user2_id) OR (C.user_id = F.user2_id AND C.user_id1 = F.user1_id))


**Solution-4:**

``` sql
SELECT a.user_id AS user_id , b.user_id AS recommended_id
FROM Listens a
JOIN Listens b
ON a.user_id <> b.user_id AND a.song_id = b.song_id and a.day = b.day
WHERE NOT EXISTS (SELECT user1_id, user2_id from friendship
WHERE (a.user_id = user1_id AND b.user_id = user2_id) OR
(a.user_id = user2_id AND b.user_id = user1_id)
)
GROUP BY a.user_id, b.user_id, a.day
HAVING COUNT(DISTINCT a.song_id)>=3